In [6]:
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfpage import PDFPage
import io

In [7]:
import os
import fitz 
file_name = "页面提取自－中行.pdf"
# file_name = "爱仕达：2018年第三季度报告正文.pdf"
# file_name = "碧水源(300070)我们真正缺少的是干净水,新水源开发破解缺水难题.pdf"
# file_name = "天龙集团：2018年第三季度报告全文 7.pdf"
full_path = os.getcwd()+"/pdfs/"+file_name
# doc = fitz.open(full_path)

In [8]:
def extract_text_from_pdf(pdf_path):
    resource_manager = PDFResourceManager()
    fake_file_handle = io.StringIO()
    converter = TextConverter(resource_manager, fake_file_handle)
    page_interpreter = PDFPageInterpreter(resource_manager, converter)
    
    with open(pdf_path, 'rb') as fh:
        for page in PDFPage.get_pages(fh, 
                                      caching=True,
                                      check_extractable=True):
            page_interpreter.process_page(page)
            
        text = fake_file_handle.getvalue()
    
    # close open handles
    converter.close()
    fake_file_handle.close()
    
    if text:
        return text

In [10]:
# extract_text_from_pdf(full_path)

In [22]:
def extract_text_by_page(pdf_path):
    with open(pdf_path, 'rb') as fh:
        for page in PDFPage.get_pages(fh, 
                                      caching=True,
                                      check_extractable=True):
            resource_manager = PDFResourceManager()
            fake_file_handle = io.StringIO()
            converter = TextConverter(resource_manager, fake_file_handle)
            page_interpreter = PDFPageInterpreter(resource_manager, converter)
            page_interpreter.process_page(page)
            
            text = fake_file_handle.getvalue()
            yield text
    
            # close open handles
            converter.close()
            fake_file_handle.close()
    
def extract_text(pdf_path):
    # for page in extract_text_by_page(pdf_path):
    #     print(page)
    #     print()
    print(list(extract_text_by_page(pdf_path))[0])


In [23]:
 extract_text(full_path)

中国银行股份有限公司  2017年12月31日合并及母公司资产负债表 (除特别注明外，金额单位均为百万元人民币)  114   中国银行集团  中国银行  注释 2017年 12月31日  2016年 12月31日  2017年 12月31日  2016年 12月31日          资产         现金及存放中央银行款项 七、1 2,303,020  2,349,188  2,191,571  2,253,714 存放同业款项 七、2 485,057  582,434  451,527  591,011 贵金属  172,763  161,417  166,687  156,155 拆出资金 七、3 486,559  483,929  512,608  461,200 以公允价值计量且其变动计入当期损益的金融资产 七、4 193,611  124,090  109,022  59,144 衍生金融资产 七、5 94,912  130,549  72,055  85,604 买入返售金融资产 七、6 88,840  110,119  73,030  92,351 应收利息 七、7 96,919  79,836  87,245  72,457 发放贷款和垫款 七、8 10,644,304  9,735,646  9,443,898  8,683,440 可供出售金融资产 七、9 1,857,222  1,609,830  1,193,010  1,026,700 持有至到期投资 七、10 2,089,864  1,843,043  2,028,333  1,773,569 应收款项类投资 七、11 414,025  395,921  402,951  378,426 长期股权投资 七、12 17,180  14,059  246,630  189,960 投资性房地产 七、13 21,026  21,659  2,025  2,144 固定资产 七、14 205,614  194,897  83,439  84,962 无形资产 七、15 18,835  14,542  17,387  13,068 商誉 七、16 2,481  2,473  -  - 递延所得税资产 七、30 46,487  34,341  47,933  35,892 持有待售资产  -

In [24]:
import os
import xml.etree.ElementTree as xml
# from miner_text_generator import extract_text_by_page
from xml.dom import minidom
def export_as_xml(pdf_path, xml_path):
    filename = os.path.splitext(os.path.basename(pdf_path))[0]
    root = xml.Element('{filename}'.format(filename=filename))
    pages = xml.Element('Pages')
    root.append(pages)
    
    counter = 1
    for page in extract_text_by_page(pdf_path):
        text = xml.SubElement(pages, 'Page_{}'.format(counter))
        text.text = page[0:100]
        counter += 1
    
    tree = xml.ElementTree(root)
    print(tree)
    xml_string = xml.tostring(root, 'utf-8')
    parsed_string = minidom.parseString(xml_string)
    pretty_string = parsed_string.toprettyxml(indent='  ')
    
    with open(xml_path, 'w') as fh:
        fh.write(pretty_string)


In [25]:
export_as_xml(full_path, xml_path="./res.xml")

ValueError: multi-byte encodings are not supported